网络层

In [1]:
import tensorflow as tf

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 简单卷积层
def conv_layer(input,channels_in,channels_out):
    weights = tf.Variable(tf.truncated_normal([5,5,channels_in,channels_out],stddev=0.05))
    biases = tf.Variable(tf.constant(0.05,shape=[channels_out]))
    conv = tf.nn.conv2d(input,filter=weights,strides=[1,1,1,1],padding='SAME')
    act = tf.nn.relu(conv + biases)
    return act
# 简化全连接层
def fc_layer(input,num_inputs,num_outpus,use_relu=True):
    weights = tf.Variable(tf.truncated_normal([num_inputs,num_outpus],stddev=0.05))
    biases = tf.Variable(tf.constant(0.05,shape=[num_outpus]))
    act = tf.matmul(input,weights) + biases
    if use_relu:
        act = tf.nn.relu(act)
    return act
# max pooling层
def max_pool(input):
    return tf.nn.max_pool(input,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

载入数据，构建网络

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('../dataset/NMIST',one_hot=True)
x = tf.placeholder(tf.float32,shape=[None,784])
y = tf.placeholder(tf.float32,shape=[None,10])
x_image = tf.reshape(x,[-1,28,28,1])
conv1 = conv_layer(x_image,1,32)
pool1 = max_pool(conv1)
conv2 = conv_layer(pool1,32,64)
pool2 = max_pool(conv2)

flat_shape = pool2.get_shape()[1:4].num_elements()
flattened = tf.reshape(pool2,[-1,flat_shape])
fc1 = fc_layer(flattened,flat_shape,1024)
logits = fc_layer(fc1,1024,10,use_relu=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../dataset/NMIST\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../dataset/NMIST\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ../dataset/NMIST\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ../dataset/NMIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as offi

交叉熵，优化器，准确率

In [5]:
# 计算交叉熵
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits))
# 使用Adam优化器来训练
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cross_entropy)
# 计算准确率
correct_prediction = tf.equal(tf.argmax(y,axis=1),tf.argmax(logits,axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



创建session,训练

In [7]:
session = tf.Session()
session.run(tf.global_variables_initializer())
train_batch_size = 100
for i in range(2000):
    x_batch,y_batch = data.train.next_batch(train_batch_size)
    feed_dict = {x:x_batch,y:y_batch}
    if i % 500 == 0:
        train_accuracy = session.run(accuracy,feed_dict=feed_dict)
        print("迭代轮次: {0:>6}, 训练准确率: {1:>6.4%}".format(i, train_accuracy))
    session.run(optimizer,feed_dict=feed_dict)

迭代轮次:      0, 训练准确率: 9.0000%
迭代轮次:    500, 训练准确率: 94.0000%
迭代轮次:   1000, 训练准确率: 99.0000%
迭代轮次:   1500, 训练准确率: 97.0000%


可视化计算图

In [9]:
import os

In [11]:
tensorboard_dir = 'tensorboard/mnist'
if not os.path.exists(tensorboard_dir):
    os.makedirs(tensorboard_dir)
writer = tf.summary.FileWriter(tensorboard_dir)
writer.add_graph(session.graph)